In [1]:
link = "https://quotes.toscrape.com/"

In [2]:
link

'https://quotes.toscrape.com/'

In [3]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [5]:
res = requests.get(link)

In [6]:
res

<Response [200]>

In [11]:
html = res.text

In [13]:
fd = open("main.html", "w",encoding="utf-8")
fd.write(html)
fd.close()

In [15]:
soup = BeautifulSoup(res.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Quotes to Scrape</title>
<link href="/static/bootstrap.min.css" rel="stylesheet"/>
<link href="/static/main.css" rel="stylesheet"/>
</head>
<body>
<div class="container">
<div class="row header-box">
<div class="col-md-8">
<h1>
<a href="/" style="text-decoration: none">Quotes to Scrape</a>
</h1>
</div>
<div class="col-md-4">
<p>
<a href="/login">Login</a>
</p>
</div>
</div>
<div class="row">
<div class="col-md-8">
<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="

In [21]:
print(soup.find("span", class_ = "text").text[1:-1])

The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.


In [26]:
for quotes in soup.find_all("span", class_ = "text"):
    print((quotes.text)[1:-1])

The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.
It is our choices, Harry, that show what we truly are, far more than our abilities.
There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.
The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.
Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.
Try not to become a man of success. Rather become a man of value.
It is better to be hated for what you are than to be loved for what you are not.
I have not failed. I've just found 10,000 ways that won't work.
A woman is like a tea bag; you never know how strong it is until it's in hot water.
A day without sunshine is like, you know, night.


In [40]:
for sp in soup.find_all("div", class_ = "quote"):
    print(sp.find("span", class_ = "text").text[1:-1])
    print("by "+sp.find("small", class_ = "author").text)
    print("https://quotes.toscrape.com/"+ sp.find("a").get("href"))

The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.
by Albert Einstein
https://quotes.toscrape.com//author/Albert-Einstein
It is our choices, Harry, that show what we truly are, far more than our abilities.
by J.K. Rowling
https://quotes.toscrape.com//author/J-K-Rowling
There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.
by Albert Einstein
https://quotes.toscrape.com//author/Albert-Einstein
The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.
by Jane Austen
https://quotes.toscrape.com//author/Jane-Austen
Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.
by Marilyn Monroe
https://quotes.toscrape.com//author/Marilyn-Monroe
Try not to become a man of success. Rather become a man of value.
by Albert Einstein
https://quotes.toscrape.com//author/

In [57]:
data = []

for i in tqdm(range(1,11)):
    res = requests.get("https://quotes.toscrape.com/page/"+str(i)+"/")
    soup = BeautifulSoup(res.text,"html.parser")
    for sp in soup.find_all('div', class_ = 'quote'):
        quote     = sp.find('span', class_ = 'text').text[1:-1]
        author    = sp.find('small').text
        author_id = sp.find('a').get('href')
        
        tags = []
        for tag in sp.find_all('a',class_ = 'tag'):
            tags.append(tag.text)
        tags      = ','.join(tags)
    
        data.append([quote, author, author_id, tags])


100%|██████████| 10/10 [00:12<00:00,  1.30s/it]


In [51]:
data

[['The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.',
  'Albert Einstein',
  '/author/Albert-Einstein',
  'change,deep-thoughts,thinking,world'],
 ['It is our choices, Harry, that show what we truly are, far more than our abilities.',
  'J.K. Rowling',
  '/author/J-K-Rowling',
  'abilities,choices'],
 ['There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.',
  'Albert Einstein',
  '/author/Albert-Einstein',
  'inspirational,life,live,miracle,miracles'],
 ['The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.',
  'Jane Austen',
  '/author/Jane-Austen',
  'aliteracy,books,classic,humor'],
 ["Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.",
  'Marilyn Monroe',
  '/author/Marilyn-Monroe',
  'be-yourself,inspirational'],
 ['Try not to become

In [53]:
df = pd.DataFrame(data, columns = ["quote","author","author_id","tags"])

In [55]:
df.to_csv("Quotes.csv", index = False)

# Book Scraping

In [4]:
data = []
for i in tqdm(range(1,51)):
    res = requests.get("https://books.toscrape.com/catalogue/page-"+str(i)+".html")
    soup = BeautifulSoup(res.text, "html.parser")
    for sp in soup.find_all("li", class_ = "col-xs-6 col-sm-4 col-md-3 col-lg-3"):
        link_for_books = "https://books.toscrape.com/catalogue/"+sp.find("div", class_ = "image_container").find("a").get("href")
        res1 = requests.get(link_for_books)
        soup1 = BeautifulSoup(res1.text,"html.parser")
        name_of_book = soup1.find("h1").text
        price_of_book = soup1.find("p", class_="price_color").text
        description_of_book = soup1.find_all("p")[3].text
        Book_code = soup1.find_all("td")[0].text
        Category = soup1.find_all("td")[1].text
        price_without_tax = soup1.find_all("td")[2].text
        price_with_tax = soup1.find_all("td")[3].text
        tax = soup1.find_all("td")[4].text
        availability = soup1.find_all("td")[5].text
        number_of_reviews = soup1.find_all("td")[6].text
        data.append([name_of_book,price_of_book,description_of_book,Book_code,Category,price_without_tax,price_with_tax,tax,availability,number_of_reviews])


 82%|████████▏ | 41/50 [23:07<05:04, 33.84s/it]


ConnectTimeout: HTTPSConnectionPool(host='books.toscrape.com', port=443): Max retries exceeded with url: /catalogue/ive-got-your-number_173/index.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002A254E9B5D0>, 'Connection to books.toscrape.com timed out. (connect timeout=None)'))

In [7]:
df = pd.DataFrame(data,columns = ["Name","Total Price","Description","Book Code","Category","Price(Without Tax)","Price(With Tax)","Tax","Availability","Number of Reviews"])

In [9]:
df.to_csv("BooksData.csv")